In [28]:
# importing the required libraries
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient import discovery

## get sheet instance

In [30]:
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('poefapp-beta-707967b7975a.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

# service to update sheet values
service = discovery.build('sheets', 'v4', credentials=creds)

sheet_id = "1zX5c5gPQJUIoHr3pLkECfvhdzofLufD-hIZzIBL3cmg"

In [14]:
# get the instance of the Spreadsheet
sheet = client.open('poeflijst')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

## testjes met acces en edits

In [19]:
# get all the records of the data
records_data = sheet_instance.get_all_records()

# view the data
records_data

[{'Persoon': 'Gidsen', 'Poef': ''},
 {'Persoon': 'Delphine', 'Poef': 5},
 {'Persoon': 'Fran', 'Poef': 7},
 {'Persoon': 'Hanne', 'Poef': 18},
 {'Persoon': 'Fien B', 'Poef': 12},
 {'Persoon': 'Alexander', 'Poef': 33},
 {'Persoon': 'Francis', 'Poef': 156341},
 {'Persoon': 'Elias', 'Poef': 25},
 {'Persoon': 'Verkenners', 'Poef': ''},
 {'Persoon': 'Emma VG', 'Poef': 1},
 {'Persoon': 'Emelie', 'Poef': 2},
 {'Persoon': 'Sam', 'Poef': 3},
 {'Persoon': 'Lander', 'Poef': 4},
 {'Persoon': 'Jakob', 'Poef': 5},
 {'Persoon': 'Anthony', 'Poef': 6},
 {'Persoon': 'Pieter', 'Poef': 7},
 {'Persoon': 'jin', 'Poef': ''},
 {'Persoon': 'Vaast', 'Poef': 1},
 {'Persoon': 'Charlotte', 'Poef': 2},
 {'Persoon': 'Kerim', 'Poef': 3}]

In [21]:
# convert the json to dataframe
records_df = pd.DataFrame.from_dict(records_data)

# view the top records
records_df

,Persoon,Poef
0,Gidsen,
1,Delphine,5
2,Fran,7
3,Hanne,18
4,Fien B,12
5,Alexander,33
6,Francis,156341
7,Elias,25
8,Verkenners,
9,Emma VG,1


In [88]:
# get the index for "Francis"
index = records_df.loc[records_df["Persoon"]=="Francis"].index[0]
print(index)
# adjust for top row removed and start of counting from 0
index += 2
range_ = "poef!B{0}".format(index)
value_range_body = {"range" : range_,
                    "values" : [[records_df["Poef"][index-2]+1]]}
# adjust the value for Francis in the sheet
request = service.spreadsheets().values().update(spreadsheetId=sheet_id, range=range_, valueInputOption="RAW", body=value_range_body)
respons = request.execute()
request.body

6


'{"range": "poef!B8", "values": [[156344]]}'

In [80]:
request.body

'{"range": "poef!B8", "values": [[156344]]}'

In [57]:
# get all the records of the data
records_data = sheet_instance.get_all_records()

# convert the json to dataframe
records_df = pd.DataFrame.from_dict(records_data)

# view the top records
records_df

,Persoon,Poef
0,Gidsen,
1,Delphine,5
2,Fran,7
3,Hanne,18
4,Fien B,12
5,Alexander,33
6,Francis,156343
7,Elias,25
8,Verkenners,
9,Emma VG,1


## update functies

In [98]:
def get_poeflijst(sheet_instance):
    """
    function to fetch the current state of the poeflijst
    
    PARAMS
    sheet_instance : a Worksheet object refering to the sheet to fetch
    
    RETURNS:
    pandas dataframe with the current state of the poeflijst
    """
    
    # get all the records of the data
    poef_JSON = sheet_instance.get_all_records()
    # convert the json to dataframe
    poef_df = pd.DataFrame.from_dict(poef_JSON)
    
    return poef_df

In [85]:
def poef(service, sheet_id, sheet_instance, name, poef_df, column="B"):
    """
    This function updates the "poef" value of person name in the sheet with 1
    
    PARAMS
    service : Type: Resource, service to interact with the google sheet
    sheet_id: String, id of the google sheet, can be found in the URL of the sheet between "d/" and "/edit"
    sheet_instance : a Worksheet object refering to the sheet to fetch
    name: String, the name of the person for whom to poef
    poef_df : DataFrame, pandas dataframe containing the values from the google sheet
    column: String, In which column to poef, default "B" !!currently not implemented!!
    
    RETURNS
    the response from the update request and the updated poef dataframe
    """
    # fetch the index corresponding to name
    index = poef_df.loc[records_df["Persoon"]==name].index[0]
    # get the current poef value
    # TODO make it possible to fetch poef from a different columnt then the first
    poef_val = poef_df["Poef"][index]
    # the cell to update in the sheet, adjust index because the df counts
    # from 0 and uses the first row of the sheet as header
    cell = "poef!{0}{1}".format(column, index+2)
    # create the body for the update
    body = {"range" : cell,
            "values" : [[poef_val+1]]}
    
    # create the update request
    request = service.spreadsheets().values().update(spreadsheetId=sheet_id, range=cell, valueInputOption="RAW", body=body)
    print(request.body)
    # execute the request, return response
    response = request.execute()
    
    # update the poef dataframe
    poef_df = get_poeflijst(sheet_instance)
    return response, poef_df

## testje

In [68]:
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name('poefapp-beta-707967b7975a.json', scope)

# authorize the clientsheet 
client = gspread.authorize(creds)

# service to update sheet values
service = discovery.build('sheets', 'v4', credentials=creds)

sheet_id = "1zX5c5gPQJUIoHr3pLkECfvhdzofLufD-hIZzIBL3cmg"

# get the instance of the Spreadsheet
sheet = client.open('poeflijst')

# get the first sheet of the Spreadsheet
sheet_instance = sheet.get_worksheet(0)

poef_df = get_poeflijst(sheet_instance)

In [72]:
poef_df.head()

,Persoon,Poef
0,Gidsen,
1,Delphine,5
2,Fran,7
3,Hanne,18
4,Fien B,12


In [103]:
response, poef_df = poef(service, sheet_id, sheet_instance, "Hanne", poef_df)
poef_df.head()

{"range": "poef!B5", "values": [[23]]}


,Persoon,Poef
0,Gidsen,
1,Delphine,5
2,Fran,7
3,Hanne,23
4,Fien B,12
